In [1]:
import numpy as np
from cvxopt import matrix
from cvxopt.solvers import qp, cp, cpl

In [11]:
def auc_amax_kfold(ps, ns, accs):
    return np.mean(1 - (1 - accs)**2*(ps + ns)**2/(2*ps*ns))

def auc_amax_kfold_solve(ps, ns, avg_acc):
    k = ps.shape[0]
    q = (ps + ns)**2/(2*ps*ns)/k*2
    Q = np.diag(q)
    A = np.repeat(1.0/k, k).reshape(-1, 1).T
    b = np.array([avg_acc])
    G = np.vstack([np.eye(k), -np.eye(k)]).astype(float)
    lower = np.array([max(p, n)/(p + n) for p, n in zip(ps, ns)])
    h = np.hstack([np.repeat(1.0, k), -lower])
    
    Q = matrix(Q)
    q = matrix(q)
    G = matrix(G)
    h = matrix(h)
    A = matrix(A)
    b = matrix(b)

    res = qp(P=Q, q=q, G=G, h=h, A=A, b=b)

    print(res['x'])

    return auc_amax_kfold(ps, ns, np.array(res['x']))

In [14]:
auc_amax_kfold_solve(
    ps = np.array([10, 11, 12]),
    ns = np.array([30, 29, 31]),
    avg_acc = 0.8
)

TypeError: auc_amax_kfold_solve() got an unexpected keyword argument 'avg_acc'

In [21]:
class F_acc_cmax:
    def __init__(self, ps, ns, avg_auc):
        self.ps = ps
        self.ns = ns
        self.avg_auc = avg_auc
        self.k = len(ps)
        self.maxs = np.array([max(p, n) for p, n in zip(ps, ns)])
        self.mins = np.array([min(p, n) for p, n in zip(ps, ns)])
        self.weights = self.mins / (ps + ns)

    def __call__(self, x=None, z=None):
        if x is None and z is None:
            return (0, matrix(1.0, (self.k, 1)))
        
        if x is not None:
            f = matrix(-np.sum(np.sqrt(x)*self.weights.reshape(-1, 1)))
            Df = matrix(-0.5/np.sqrt(x)*self.weights.reshape(-1, 1)).T
        
        if z is None:
            return (f, Df)
        
        hess = np.diag(0.5**2*np.array(x)[:, 0]**(-3/2)*self.weights)
        
        hess = matrix(z[0]*hess)

        return (f, Df, hess)


def acc_cmax_kfold(ps, ns, aucs):
    maxs = np.array([max(p, n) for p, n in zip(ps, ns)])
    mins = np.array([min(p, n) for p, n in zip(ps, ns)])

    return np.mean((maxs + mins * np.sqrt(2*aucs - 1)) / (ps + ns))

def acc_cmax_kfold_solve(ps, ns, avg_auc):
    F = F_acc_cmax(ps, ns, avg_auc)

    k = ps.shape[0]
    q = (ps + ns)**2/(2*ps*ns)/k*2
    Q = np.diag(q)
    A = np.repeat(1.0/k, k).reshape(-1, 1).T
    b = np.array([2*avg_auc - 1])
    G = np.vstack([np.eye(k), -np.eye(k)]).astype(float)
    h = np.hstack([np.repeat(1.0, k), -np.repeat(0, k)])
    
    Q = matrix(Q)
    q = matrix(q)
    G = matrix(G)
    h = matrix(h)
    A = matrix(A)
    b = matrix(b)

    res = cp(F, G, h, A=A, b=b)

    aucs = (np.array(res['x']) + 1)/2

    print(aucs)

    return acc_cmax_kfold(ps, ns, aucs)

In [22]:
acc_cmax_kfold_solve(
    ps = np.array([10, 11, 12]),
    ns = np.array([30, 29, 31]),
    avg_auc = 0.8
)

     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -3.8041e+00  7e+00  1e+00  2e-01
 1: -6.3352e-01 -2.0921e+00  1e+00  1e-02  2e-02
 2: -6.2321e-01 -6.5391e-01  3e-02  2e-04  4e-04
 3: -6.2378e-01 -6.2797e-01  5e-03  2e-04  5e-04
 4: -6.2360e-01 -6.2383e-01  3e-04  3e-05  1e-04
 5: -6.2355e-01 -6.2355e-01  4e-06  5e-07  2e-06
 6: -6.2355e-01 -6.2355e-01  4e-08  5e-09  2e-08
Optimal solution found.
[[0.76041082]
 [0.81509697]
 [0.82449221]]


np.float64(0.9393483904285505)

In [24]:
class F_acc_amin:
    def __init__(self, ps, ns, avg_auc):
        self.ps = ps
        self.ns = ns
        self.avg_auc = avg_auc
        self.k = len(ps)
        self.maxs = np.array([max(p, n) for p, n in zip(ps, ns)])
        self.mins = np.array([min(p, n) for p, n in zip(ps, ns)])
        self.weights = np.sqrt(ps*ns)/(ps + ns)

    def __call__(self, x=None, z=None):
        if x is None and z is None:
            return (0, matrix(1.0, (self.k, 1)))
        
        if x is not None:
            f = matrix(-np.sum(np.sqrt(x)*self.weights.reshape(-1, 1)))
            Df = matrix(-0.5/np.sqrt(x)*self.weights.reshape(-1, 1)).T
        
        if z is None:
            return (f, Df)
        
        hess = np.diag(0.5**2*np.array(x)[:, 0]**(-3/2)*self.weights)
        
        hess = matrix(z[0]*hess)

        return (f, Df, hess)

def acc_amin_kfold(ps, ns, aucs):
    maxs = np.array([max(p, n) for p, n in zip(ps, ns)])
    mins = np.array([min(p, n) for p, n in zip(ps, ns)])

    return np.mean(1 - np.sqrt(2*(1 - aucs)*ps*ns)/(ps + ns))

def acc_amin_kfold_solve(ps, ns, avg_auc):
    F = F_acc_cmax(ps, ns, avg_auc)

    lower_bounds = 2*(1 - 1.0 - F.mins/(2*F.maxs))

    k = ps.shape[0]
    q = (ps + ns)**2/(2*ps*ns)/k*2
    Q = np.diag(q)
    A = np.repeat(1.0/k, k).reshape(-1, 1).T
    b = np.array([2*(1 - avg_auc)])
    G = np.vstack([np.eye(k), -np.eye(k)]).astype(float)
    h = np.hstack([np.repeat(1.0, k), -lower_bounds])
    
    Q = matrix(Q)
    q = matrix(q)
    G = matrix(G)
    h = matrix(h)
    A = matrix(A)
    b = matrix(b)

    res = cp(F, G, h, A=A, b=b)

    aucs = 1 - (np.array(res['x']))/2

    print(aucs)

    return acc_amin_kfold(ps, ns, aucs)

In [25]:
acc_amin_kfold_solve(
    ps = np.array([10, 11, 12]),
    ns = np.array([30, 29, 31]),
    avg_auc = 0.8
)

     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -4.9038e+00  7e+00  1e+00  2e-01
 1: -5.5428e-01 -1.8009e+00  1e+00  3e-02  7e-02
 2: -5.0996e-01 -5.4869e-01  4e-02  8e-04  2e-03
 3: -5.0961e-01 -5.1314e-01  4e-03  3e-04  1e-03
 4: -5.0914e-01 -5.0922e-01  1e-04  1e-05  5e-05
 5: -5.0912e-01 -5.0912e-01  1e-06  1e-07  5e-07
 6: -5.0912e-01 -5.0912e-01  1e-08  1e-09  5e-09
Optimal solution found.
[[0.82639285]
 [0.78993535]
 [0.7836718 ]]


np.float64(0.7203398354097179)